# Intro

Let's play Blackjack!

![Blackjack](http://www.hightechgambling.com/sites/default/files/styles/large/public/casino/table_games/blackjack.jpg)

# Setting up the environment and collecting data


In [ ]:
#WL - Empty function.  
def should_hit(player_total, dealer_card_val, player_aces):
    """Return True if the player should hit (request another card) given the current game
    state, or False if the player should stay. player_aces is the number of aces the player has.
    """
    return False

In [ ]:
# SETUP. You don't need to worry for now about what this code does or how it works. 
from learntools.core import binder; binder.bind(globals())
from learntools.python.ex3 import q7 as blackjack
print('Setup complete.')

In [ ]:
from contextlib import redirect_stdout
from io import StringIO
import numpy as np
def simulate_game(verbose=False):
    out_buffer = StringIO()
    
    with redirect_stdout(out_buffer):
        blackjack.simulate_one_game()
    out_str = out_buffer.getvalue()
    #print("william",out_str)
    if verbose: 
        print("verbose",out_str)
    return any(['Player wins' in x for x in out_str.split('\n')]) # if any lines say player wins then we won
#simulate_game(True)
simulate_game(False)

In [ ]:
global val_list
val_list = []
def should_hit(player_total, dealer_card_val, player_aces):
    global val_list
    cur_move = np.random.choice([True, False])
    val_list+=[(player_total, dealer_card_val, player_aces, cur_move)]
    return cur_move

simulate_game(False)
#print(val_list)

In [ ]:
%matplotlib inline
import pandas as pd
out_rows=[]
for i in range(175000):
    val_list=[]
    c_score = simulate_game(False)
    #print("william",c_score)
    for i, (player_total, dealer_card_val, player_aces, cur_move) in enumerate(reversed(val_list)):
        score = 1.0*c_score if i==0 else 0.4+0.1*c_score
        out_rows+=[{'result': score,
                    'player_total': player_total, 
                    'dealer_card_val': dealer_card_val, 
                    'player_aces': player_aces, 
                    'cur_move': cur_move,
                    'winning_hand': c_score
                   }]
move_df = pd.DataFrame(out_rows)
print('Win Percentages: {:2.2f}'.format(move_df[move_df['result'].isin([0.0, 1.0])]['result'].mean()*100))
move_df.to_csv('all_games.csv', index=False) # save the results
#move_df.head(10)

In [ ]:
#WL - get winning moves only
criteria = move_df['result']>=0.5
move_df = move_df[criteria]
#move_df.describe()
#move_df.head(10)

# Build different models

In [ ]:
#build the model (Logistic Regression)
X = move_df[["dealer_card_val","player_aces","player_total"]]
#print(X.head())
Y = move_df["cur_move"]
#print(Y.describe())
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')
lr = lr.fit(X,Y)

In [ ]:
#WL - build model (Naive Bayes)
X = move_df[["dealer_card_val","player_aces","player_total"]]
Y = move_df["cur_move"]
from sklearn.naive_bayes import GaussianNB
NB_classifier = GaussianNB()
NB_classifier.fit(X,Y)

In [ ]:
#WL - build model (Random Forest Classifier)
X = move_df[["dealer_card_val","player_aces","player_total"]]
Y = move_df["cur_move"]
from sklearn.ensemble import RandomForestClassifier
RF_classifier = RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=0)
RF_classifier.fit(X,Y)

In [ ]:
#WL - build model (Neural Network)
X = move_df[['dealer_card_val','player_aces','player_total']]
Y = move_df['cur_move']
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(solver='lbfgs',alpha=1e-5, hidden_layer_sizes=(5,2), random_state=1)
nn.fit(X,Y)

# Test the models

In [ ]:
#WL - test the model (Neural Network)
prediction = nn.predict([[1,0,18]])
print("prediction (expect False)=",prediction)

prediction = nn.predict([[8,0,8]])
print("prediction (expect True)=",prediction)

In [ ]:
#WL - test the model (Naive Bayes)
prediction = NB_classifier.predict([[1,0,18]])
print("prediction (expect False)=",prediction)

prediction = NB_classifier.predict([[8,0,8]])
print("prediction (expect True)=",prediction)

In [ ]:
#WL - test the model (Random Forest Classifier)
prediction = RF_classifier.predict([[1,0,18]])
print("prediction (expect False)=",prediction)

prediction = RF_classifier.predict([[8,0,8]])
print("prediction (expect True)=",prediction)

In [ ]:
#WL - Test the model (Logistic Regression)
prediction_lr = lr.predict([[1,0,18]])
print("prediction (expect False)=",prediction_lr)

prediction_lr = lr.predict([[8,0,8]])
print("prediction (expect True)=",prediction_lr)

# Simulate Blackjack games using different models

In [ ]:
#build the decision making (Logistic Regression)
def should_hit(player_total, dealer_card_val, player_aces):
    prediction_lr = lr.predict([[dealer_card_val,player_aces,player_total]])
    return prediction_lr

print("Logistic Regression Model")
blackjack.simulate(n_games=10000)


In [ ]:
#build the decision making (Naive Bayes)
def should_hit(player_total, dealer_card_val, player_aces):
    prediction= NB_classifier.predict([[dealer_card_val,player_aces,player_total]])
    return prediction
print("Naive Bayes Model")
blackjack.simulate(n_games=10000)

In [ ]:
#build the decision making (Random Forest Classifier)
def should_hit(player_total, dealer_card_val, player_aces):
    prediction= RF_classifier.predict([[dealer_card_val,player_aces,player_total]])
    return prediction
print("Random Forest Model")
blackjack.simulate(n_games=1000)

In [ ]:
#WL - build the decision making (Neural Network)
def should_hit(player_total, dealer_card_val, player_aces):
    prediction= nn.predict([[dealer_card_val,player_aces,player_total]])
    return prediction
print("Neural Network Model")
blackjack.simulate(n_games=10000)